# SDK Connection Test

This notebook tests basic SDK functionality before running the full lab.

**Note:** You may see an "unverified HTTPS request" warning - this can be safely ignored.

## Step 0: Reinstall SDK Package

This ensures you have a clean installation of the SDK.

In [ ]:
!pip uninstall ibm_watsonx_data_integration -y
!pip install ibm_watsonx_data_integration --force-reinstall

## Step 1: Test SDK Import

In [ ]:
# Import required modules
try:
    from ibm_watsonx_data_integration import *
    from ibm_watsonx_data_integration.common.auth import IAMAuthenticator
    from ibm_watsonx_data_integration.services.datastage import *
    print("✓ SDK imported successfully")
except ImportError as e:
    print(f"✗ Import failed: {e}")

## Step 2: Set Credentials

Replace with your actual credentials.

**TIP**: Create a `credentials.txt` file on your desktop with this format to make copy/paste easier:
```
API_KEY = "your-api-key-here"
PROJECT_ID = "your-project-id-here"
```

In [ ]:
API_KEY = "YOUR_IBM_CLOUD_API_KEY"
PROJECT_ID = "YOUR_PROJECT_ID"

print("Credentials set (not displayed for security)")

## Step 3: Create Authentication

In [ ]:
try:
    auth = IAMAuthenticator(api_key=API_KEY, base_auth_url="https://cloud.ibm.com")
    print("✓ Authenticator created")
except Exception as e:
    print(f"✗ Authentication failed: {e}")

## Step 4: Connect to Platform

In [ ]:
try:
    platform = Platform(auth, base_api_url="https://api.ca-tor.dai.cloud.ibm.com")
    print("✓ Platform connection initialized")
except Exception as e:
    print(f"✗ Platform connection failed: {e}")

## Step 5: Get Project

**WAIT FOR THIS STEP** - Make sure you see the success message before continuing!

In [ ]:
try:
    project = platform.projects.get(guid=PROJECT_ID)
    print(f"✓ Successfully connected to project: {project.name}")
    print(f"  Project ID: {PROJECT_ID}")
    print("\n✓✓✓ CONFIRMATION: You are ready to proceed to the next step!")
except Exception as e:
    print(f"✗ Failed to get project: {e}")

## Step 6: Create a Simple Flow (Row Generator → Peek)

Now let's create a basic DataStage flow to verify everything works end-to-end.

In [ ]:
# Create flow
flow = project.create_flow(name="RowGenPeek_SDK_Test", environment=None, flow_type="datastage")
print("✓ Flow created")

# Add Row Generator stage
row_generator = flow.add_stage("Row Generator", "Row_Generator")
row_generator.configuration.runtime_column_propagation = False
print("✓ Added Row Generator stage")

# Add Peek stage
peek = flow.add_stage("Peek", "Peek")
peek.configuration.runtime_column_propagation = False
print("✓ Added Peek stage")

# Connect stages (create the link between them)
link_1 = row_generator.connect_output_to(peek)
link_1.name = "Link_1"
print("✓ Connected stages")

# Define schema (what data flows through the link)
row_generator_schema = link_1.create_schema()
row_generator_schema.add_field("VARCHAR", "COLUMN_1").length(100)
print("✓ Schema defined")

# Update flow in watsonx.data Integration
project.update_flow(flow)
print("✓ Flow saved to watsonx.data Integration")

# Create a job to run the flow
row_gen_peek_job = project.create_job(name="RowGenPeek_job", flow=flow)
print("✓ Job created")

# Start the job
row_gen_peek_job_run = row_gen_peek_job.start(name="RowGenPeek_job_run", description="")
print("✓ Job started")

print("\n" + "="*60)
print("✓✓✓ SUCCESS! Your flow and job have been created!")
print("="*60)
print("\nGo to the watsonx.data Integration UI to see your work:")
print("https://ca-tor.dai.cloud.ibm.com/df/home?context=df")
print("\nNavigation: Sandbox project → Assets tab")
print("\nYou should see:")
print("  • Flow: RowGenPeek_SDK_Test")
print("  • Job: RowGenPeek_job (currently running)")

## Summary

If all cells above show ✓ marks, your environment is ready!

**What you just did:**
- ✓ Imported the SDK
- ✓ Authenticated with IBM Cloud
- ✓ Connected to your project
- ✓ Created a simple DataStage flow programmatically

**Next:** Check the UI to see your flow, then proceed to the main lab notebooks where you'll create more complex data integration pipelines.